In [ ]:
"""
Autor: Laura Santiago
"""

from typing import List, Tuple
import numpy as np


class QuantumState:
    #Representa un ket (vector columna) en espacio finito de dimensión N.
    #Se garantiza que el ket está normalizado al crear la instancia.

    def __init__(self, amplitudes: np.ndarray):
        amp = np.asarray(amplitudes, dtype=np.complex128).reshape(-1)
        norm = np.linalg.norm(amp)
        if norm == 0:
            raise ValueError("El vector ket no puede ser el vector cero")
        self._vec = amp / norm

    @property
    def vec(self) -> np.ndarray:
        return self._vec

    @property
    def dim(self) -> int:
        return self._vec.size

# 1. Probabilidad de encontrar la partícula en la posición (índice) dada.
    def probability_of_position(self, position: int) -> float:

        if position < 0 or position >= self.dim:
            raise IndexError("posición fuera de rango")
        amp = self._vec[position]
        return float(np.real_if_close(np.vdot(amp, amp)))

    def probabilities(self) -> np.ndarray:
        """Vector de probabilidades por cada posición."""
        return np.abs(self._vec) ** 2

# 2. Probabilidad de transitar del estado self al estado other tras una medición.
    def transition_probability(self, other: 'QuantumState') -> float:

        if self.dim != other.dim:
            raise ValueError("Dimensiones incompatibles")
        amp = np.vdot(other.vec, self.vec)
        return float(np.abs(amp) ** 2)

    def inner_product(self, other: 'QuantumState') -> complex:
        if self.dim != other.dim:
            raise ValueError("Dimensiones incompatibles")
        return np.vdot(self.vec, other.vec)

Retos de programacion del capitulo 4.

In [ ]:

class QuantumSystem:

    def __init__(self, N: int):
        if N <= 0:
            raise ValueError("N debe ser positivo")
        self.N = int(N)

    def create_ket(self, amplitudes: List[complex]) -> QuantumState:
        arr = np.asarray(amplitudes, dtype=np.complex128)
        if arr.size != self.N:
            raise ValueError(f"El ket debe tener dimensión {self.N}")
        return QuantumState(arr)

    # 1) Probabilidad de encontrar en posición particular
    def probability_position(self, ket: QuantumState, position: int) -> float:
        return ket.probability_of_position(position)

    # 2) Probabilidad de transición entre dos kets
    def transition_probability(self, ket1: QuantumState, ket2: QuantumState) -> float:
        return ket1.transition_probability(ket2)

    # Retos capítulo 4
    # 1. Amplitud de transición (ya definido en QuantumState)

    # 2. Observable: comprobar hermiticidad; calcular media y varianza
    def is_hermitian(self, mat: np.ndarray, tol: float = 1e-10) -> bool:
        A = np.asarray(mat, dtype=np.complex128)
        if A.shape != (self.N, self.N):
            raise ValueError(f"Matriz debe ser {self.N}x{self.N}")
        return np.allclose(A, A.conj().T, atol=tol)

    def expectation(self, observable: np.ndarray, ket: QuantumState) -> complex:
        A = np.asarray(observable, dtype=np.complex128)
        if A.shape != (self.N, self.N):
            raise ValueError("Dimensiones incompatibles para el observable")
        if not self.is_hermitian(A):
            raise ValueError("El observable no es hermitiano")
        psi = ket.vec
        return float(np.vdot(psi, A.dot(psi)))

    def variance(self, observable: np.ndarray, ket: QuantumState) -> float:
        exp = self.expectation(observable, ket)
        psi = ket.vec
        A = np.asarray(observable, dtype=np.complex128)
        # var = <A^2> - <A>^2
        A2 = A.dot(A)
        exp2 = np.vdot(psi, A2.dot(psi))
        var = exp2 - exp * exp
        # var debe ser real, pero puede tener ruido numérico
        return float(np.real_if_close(var))

    # 3. Valores propios y probabilidad de colapso a vectores propios
    def spectral_decomposition(self, observable: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        A = np.asarray(observable, dtype=np.complex128)
        if A.shape != (self.N, self.N):
            raise ValueError("Dimensiones incompatibles")
        if not self.is_hermitian(A):
            raise ValueError("El observable no es hermitiano")
        # eigh asegura vectores ortonormales para hermitianas
        vals, vecs = np.linalg.eigh(A)
        return vals, vecs  # vecs columnas son vectores propios

    def collapse_probabilities_to_eigenbasis(self, observable: np.ndarray, ket: QuantumState) -> Tuple[np.ndarray, np.ndarray]:
        vals, vecs = self.spectral_decomposition(observable)
        psi = ket.vec
        # calcular |<eigen_i | psi>|^2
        probs = np.array([np.abs(np.vdot(vecs[:, i], psi)) ** 2 for i in range(self.N)])
        return vals, probs

    # 4. Dinámica: aplicar serie de matrices Un
    def evolve(self, ket: QuantumState, unitaries: List[np.ndarray]) -> QuantumState:
        psi = ket.vec
        for U in unitaries:
            U = np.asarray(U, dtype=np.complex128)
            if U.shape != (self.N, self.N):
                raise ValueError("Unidadarios deben ser NxN")
            # opcional: chequear unitariedad
            if not np.allclose(U.conj().T.dot(U), np.eye(self.N), atol=1e-8):
                raise ValueError("Uno de los operadores no es unitario (dentro de tolerancia)")
            psi = U.dot(psi)
        return QuantumState(psi)


Ejemplos


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Ejemplo 1: Probabilidad de colapso y media/varianza
def ejemplo_4_3_1():
    print("\nEjemplo 4.3.1 — Probabilidades post-medición")

    sistema = QuantumSystem(2)
    ket = sistema.create_ket([1/np.sqrt(2), 1/np.sqrt(2)])  # estado |+>
    observable = np.array([[1, 0],
                           [0, -1]], dtype=complex)  # Pauli Z

    valores, probs = sistema.collapse_probabilities_to_eigenbasis(observable, ket)

    print("Autovalores del observable:", valores)
    print("Probabilidades de colapso:", probs)
    print("Suma de probabilidades:", probs.sum())

    media = sistema.expectation(observable, ket)
    var = sistema.variance(observable, ket)
    print(f"Media esperada = {media:.3f}")
    print(f"Varianza = {var:.3f}")


# Ejemplo 2: Distribución de probabilidad con gráfica
def ejemplo_4_3_2():
    print("\n Ejemplo 4.3.2 — Distribución de probabilidad")

    sistema = QuantumSystem(2)
    ket = sistema.create_ket([np.sqrt(3)/2, 1/2])
    observable = np.array([[0, 1],
                           [1, 0]], dtype=complex)  # Pauli X

    vals, probs = sistema.collapse_probabilities_to_eigenbasis(observable, ket)

    plt.bar([str(round(v, 2)) for v in vals], probs, color='mediumslateblue')
    plt.title("Distribución de probabilidad (observable X)")
    plt.xlabel("Autovalores")
    plt.ylabel("Probabilidad")
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.show()

    media = sistema.expectation(observable, ket)
    var = sistema.variance(observable, ket)
    print(f"Media esperada = {media:.3f}")
    print(f"Varianza = {var:.3f}")


# Ejemplo 3: Verificación de matrices unitarias
def ejemplo_4_4_1():
    print("\n Ejemplo 4.4.1 — Verificación de matrices unitarias")

    sistema = QuantumSystem(2)

    U1 = np.array([[0, 1],
                   [1, 0]], dtype=complex)

    U2 = np.array([[np.sqrt(2)/2, np.sqrt(2)/2],
                   [np.sqrt(2)/2, -np.sqrt(2)/2]], dtype=complex)  # Hadamard

    def es_unitaria(M):
        return np.allclose(M.conj().T @ M, np.eye(M.shape[0]))

    print("U1 es unitaria:", es_unitaria(U1))
    print("U2 es unitaria:", es_unitaria(U2))
    print("U1 * U2 es unitaria:", es_unitaria(U1 @ U2))


# Ejemplo 4: Dinámica temporal del sistema
def ejemplo_4_4_2():
    print("\n Ejemplo 4.4.2 — Dinámica temporal")

    sistema = QuantumSystem(4)
    psi0 = sistema.create_ket([1, 0, 0, 0])

    i = 1j
    U = np.array([
        [0, 1/np.sqrt(2), 1/np.sqrt(2), 0],
        [i/np.sqrt(2), 0, 0, 1/np.sqrt(2)],
        [1/np.sqrt(2), 0, 0, i/np.sqrt(2)],
        [0, 1/np.sqrt(2), -1/np.sqrt(2), 0]
    ], dtype=complex)

    pasos = [U, U, U]  # aplicar tres veces la misma matriz
    estados = [psi0]
    estado_actual = psi0

    for paso in pasos:
        estado_actual = sistema.evolve(estado_actual, [paso])
        estados.append(estado_actual)

    for t, psi_t in enumerate(estados):
        print(f"\nPaso t = {t}")
        print("Estado:", psi_t.vec)
        print("Probabilidades:", np.abs(psi_t.vec)**2)

    print("\nProbabilidad de encontrar la partícula en la posición 3:")
    for t, psi_t in enumerate(estados):
        p3 = np.abs(psi_t.vec[2])**2
        print(f"t = {t}: P(posición 3) = {p3:.4f}")


# Ejecución de todos los ejemplos
if __name__ == "__main__":
    ejemplo_4_3_1()
    ejemplo_4_3_2()
    ejemplo_4_4_1()
    ejemplo_4_4_2()


Discusión de los ejercicios 4.5.2 y 4.5.3

Los ejercicios 4.5.2 y 4.5.3 tratan sobre dos ideas fundamentales de la mecánica cuántica que también son esenciales en la computación cuántica: la superposición y la medición.

En primer lugar, el ejercicio 4.5.2 explora cómo un sistema cuántico puede encontrarse en una superposición de varios estados al mismo tiempo. Por ejemplo, un qubit no está simplemente en “0” o en “1”, sino que puede estar en una combinación de ambos estados, expresada matemáticamente como

$∣𝜓⟩=𝛼∣0⟩+𝛽∣1⟩$

donde los coeficientes complejos
𝛼 y β representan las amplitudes de probabilidad de cada estado.
La suma de los cuadrados de los módulos de estas amplitudes siempre debe ser 1, es decir:

$∣𝛼∣^{2}+∣𝛽∣^{2}=1$

Este principio de superposición permite que los sistemas cuánticos procesen mucha más información que los sistemas clásicos, ya que pueden representar múltiples posibilidades simultáneamente. En el código del simulador, esto se ve cuando un vector de estado tiene varias amplitudes distintas de cero, indicando que el sistema “vive” en más de un estado a la vez.

Por otro lado, el ejercicio 4.5.3 se enfoca en el proceso de medición. Cuando se mide un sistema cuántico, la superposición desaparece y el estado “colapsa” a uno de los resultados posibles. La probabilidad de obtener un resultado específico depende del cuadrado del módulo de la amplitud correspondiente.
Por ejemplo, si el sistema estaba en el estado

$∣𝜓⟩=1/2^{1/2}(∣0⟩+∣1⟩)$,
entonces la probabilidad de medir ∣0⟩ o ∣1⟩ es igual (50% cada una).

Estos ejercicios también muestran cómo las operaciones unitarias, como la puerta de Hadamard, pueden transformar los estados. Esta puerta convierte un estado base como ∣0⟩ en una superposición equilibrada, y al aplicarla nuevamente, devuelve el sistema a su estado inicial. Esto demuestra cómo la mecánica cuántica permite crear y manipular superposiciones de forma controlada, algo que no tiene equivalente en la computación clásica.

En conclusión, los ejercicios 4.5.2 y 4.5.3 nos ayudan a comprender que:

La superposición es lo que permite que un sistema cuántico contenga múltiples posibilidades simultáneamente.

La medición transforma esa superposición en un resultado clásico con una probabilidad determinada.

Las operaciones unitarias controlan cómo evoluciona el estado cuántico sin perder información.

Estas ideas forman la base del funcionamiento de los algoritmos cuánticos, donde se aprovechan la superposición y la interferencia para obtener resultados más eficientes que los métodos clásicos.